# Chapter 3

In [1]:
using ControlSystems

## 伝達関数モデルの記述

In [2]:
Np = [0, 1]      # 伝達関数の分子多項式の係数 (0*s + 1)
Dp = [1, 2, 3]   # 伝達関数の分母多項式の係数 (1*s^2 + 2*s + 3)
P = tf(Np, Dp)

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
     1
------------
s^2 + 2s + 3

Continuous-time transfer function model

In [3]:
P = tf([0, 1], [1, 2, 3])

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
     1
------------
s^2 + 2s + 3

Continuous-time transfer function model

### 練習問題

In [4]:
P = tf([1, 2], [1, 5, 3, 4])

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
       s + 2
-------------------
s^3 + 5s^2 + 3s + 4

Continuous-time transfer function model

分母多項式の展開

In [5]:
using Symbolics
@variables s
expand( (s+1)*(s+2)^2 )

4 + s^3 + 8s + 5(s^2)

In [6]:
P = tf([1, 3],[1, 5, 8, 4])

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
       s + 3
-------------------
s^3 + 5s^2 + 8s + 4

Continuous-time transfer function model

In [7]:
P1 = tf([1, 3], [0, 1])
P2 = tf([0, 1], [1, 1])
P3 = tf([0, 1], [1, 2])
P = P1 * P2 * P3^2

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
       s + 3
-------------------
s^3 + 5s^2 + 8s + 4

Continuous-time transfer function model

In [8]:
s = tf("s")
P = 1/(s+1)/(s+2)^2

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
         1
-------------------
s^3 + 5s^2 + 8s + 4

Continuous-time transfer function model

### 分母・分子多項式の係数の抽出

In [9]:
den(P)

1×1 Matrix{Vector{Int64}}:
 [1, 5, 8, 4]

In [10]:
num(P)

1×1 Matrix{Vector{Int64}}:
 [1]

## 状態空間モデルの記述

In [11]:
A = [0 1; -1 -1]
B = [0; 1]
C = [1 0]
D = 0
P = ss(A, B, C, D)

StateSpace{Continuous, Int64}
A = 
  0   1
 -1  -1
B = 
 0
 1
C = 
 1  0
D = 
 0

Continuous-time state-space model

### 練習問題

In [12]:
A = [1 1 2; 2 1 1; 3 4 5]
B = [2; 0; 1]
C = [1 1 0]
D = 0
P = ss(A, B, C, D)

StateSpace{Continuous, Int64}
A = 
 1  1  2
 2  1  1
 3  4  5
B = 
 2
 0
 1
C = 
 1  1  0
D = 
 0

Continuous-time state-space model

### A,B,C,D行列の抽出

In [13]:
println("A=", P.A)
println("B=", P.B)
println("C=", P.C)
println("D=", P.D)

A=[1 1 2; 2 1 1; 3 4 5]
B=[2; 0; 1;;]
C=[1 1 0]
D=[0;;]


In [14]:
sysA, sysB, sysC, sysD = ssdata(P)
println("A=", sysA)
println("B=", sysB)
println("C=", sysC)
println("D=", sysD)

A=[1 1 2; 2 1 1; 3 4 5]
B=[2; 0; 1;;]
C=[1 1 0]
D=[0;;]


## ブロック線図の結合

In [15]:
S1 = tf( [0, 1], [1, 1])

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
  1
-----
s + 1

Continuous-time transfer function model

In [16]:
S2 = tf( [1, 1], [1, 1, 1])

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
   s + 1
-----------
s^2 + s + 1

Continuous-time transfer function model

### 直列結合

In [17]:
S = S2 * S1;
println(S)

S = series(S1, S2);
println(S)

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
       s + 1
-------------------
s^3 + 2s^2 + 2s + 1

Continuous-time transfer function model
TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
       s + 1
-------------------
s^3 + 2s^2 + 2s + 1

Continuous-time transfer function model


分母分子の共通因子 s+1 が約分されない
この場合は，minreal を使う

In [18]:
minreal(S)

TransferFunction{Continuous, ControlSystems.SisoRational{Float64}}
                       1.0
-------------------------------------------------
1.0s^2 + 1.0000000000000004s + 0.9999999999999992

Continuous-time transfer function model

### 並列結合

In [19]:
S = S1 + S2;
println(S)

S = parallel(S1, S2)
println(S)

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
   2s^2 + 3s + 2
-------------------
s^3 + 2s^2 + 2s + 1

Continuous-time transfer function model
TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
   2s^2 + 3s + 2
-------------------
s^3 + 2s^2 + 2s + 1

Continuous-time transfer function model


### フィードバック結合

In [20]:
S = S1*S2 / (1 + S1*S2)
println(S)

S = feedback(S1*S2, 1)
println(S)

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
        s^4 + 3s^3 + 4s^2 + 3s + 1
------------------------------------------
s^6 + 4s^5 + 9s^4 + 13s^3 + 12s^2 + 7s + 2

Continuous-time transfer function model
TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
       s + 1
-------------------
s^3 + 2s^2 + 3s + 2

Continuous-time transfer function model


In [21]:
minreal(S)

TransferFunction{Continuous, ControlSystems.SisoRational{Float64}}
                       1.0
-------------------------------------------------
1.0s^2 + 0.9999999999999993s + 2.0000000000000018

Continuous-time transfer function model

### 練習問題

In [22]:
S1 = tf(1, [1, 1])
S2 = tf(1, [1, 2])
S3 = tf([3, 1], [1, 0])
S4 = tf([2, 0], [0, 1])

S12 = feedback(S1, S2)
S123 = series(S12, S3)
S = feedback(S123, S4)

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
  3s^2 + 7s + 2
-----------------
7s^3 + 17s^2 + 7s

Continuous-time transfer function model

## 補遺（実現問題とプロパー性）

### 実現問題

In [23]:
P = tf( [0, 1], [1, 1, 1])

Pss = ss(P) # 伝達関数モデルから状態空間モデルへの変換
println(Pss)
Ptf = tf(Pss) # 状態空間モデルから伝達関数モデルへの変換
println(Ptf)

StateSpace{Continuous, Float64}
A = 
  0.0   1.0
 -1.0  -1.0
B = 
 0.0
 1.0
C = 
 1.0  0.0
D = 
 0.0

Continuous-time state-space model
TransferFunction{Continuous, ControlSystems.SisoRational{Float64}}
  1.1102230246251565e-16s + 1.0000000000000004
-------------------------------------------------
1.0s^2 + 0.9999999999999999s + 0.9999999999999996

Continuous-time transfer function model


### プロパー性

In [24]:
S1 = tf([1, 1], [0, 1])
S2 = tf([0, 1], [1, 1])

S = series(S1, S2)
minreal(S)

TransferFunction{Continuous, ControlSystems.SisoRational{Float64}}
1.0
---
1.0

Continuous-time transfer function model

In [25]:
print(S2)
ss(S2)

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
  1
-----
s + 1

Continuous-time transfer function model

StateSpace{Continuous, Float64}
A = 
 -1.0
B = 
 1.0
C = 
 1.0
D = 
 0.0

Continuous-time state-space model

In [26]:
print(S1)
ss(S1)

TransferFunction{Continuous, ControlSystems.SisoRational{Int64}}
s + 1
-----
  1

Continuous-time transfer function model

LoadError: System is improper, a state-space representation is impossible